In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
# Scenarios to plot in format:
# <Label (as included in the figure)>: <scenario name as listed in config.yaml file and included in results.csv>
# assumes "-20%" scenarios have the scenario name and "+20%" scenarios have the same name postfixed with a "p"
scenarios = {
    "reference": "default",  # used for calculations; removed before plot is made
    "WACC": "sensitivity-1",
    "Import volume": "sensitivity-2",
    "Domestic demand": "sensitivity-3",
    "Invest RES": "sensitivity-4",
    "Invest battery": "sensitivity-5",
    "Invest electrolysis": "sensitivity-6",
    "Invest MeOH syn": "sensitivity-7",
    "Invest pipeline": "sensitivity-8",
}

scenarios_name = {v: k for k, v in scenarios.items()}

# Remove scenarios which are not relevant (esc specific)
# if the specific ESC is not plotted
if snakemake.wildcards["esc"] == "shipping-meoh":
    scenarios.pop("Invest pipeline")
elif snakemake.wildcards["esc"] == "pipeline-h2":
    scenarios.pop("Invest MeOH syn")

In [ ]:
# Select data of interest
df = pd.read_csv(snakemake.input[0], sep=";")

df = df[
    (df["year"] == int(snakemake.wildcards["year"]))
    & (df["esc"] == snakemake.wildcards["esc"])
    & (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["importer"] == snakemake.wildcards["importer"])
    & (df["subcategory"] == "Cost per MWh delivered")
]

df = df.set_index("scenario")

In [ ]:
## Prepare dataframe with results

# used calculating % difference of scenarios
# pop as the "reference" is no longer needed
reference_value = df.loc[scenarios.pop("reference")]["value"]

# lower values = -20% scenario value results
lower_values = df.loc[df.index.isin(scenarios.values())]["value"]
lower_values.name = "-20%"

# Upper value scenarios have same scenario number but with an postfix "p"
upper_values = df.loc[df.index.isin([v + "p" for v in scenarios.values()])]["value"]
upper_values.name = "+20%"

# Same index names
upper_values.index = upper_values.index.str.replace("p", "")

# Combine all values into single df (two columns)
tdf = lower_values.to_frame()
df = pd.merge(tdf, right=upper_values, left_index=True, right_index=True)

df["label"] = [scenarios_name[idx] for idx in df.index]

df = df.reset_index()  # Needed for yticklabel positions
df = df.sort_values("-20%", ascending=True)

In [ ]:
## Plotting

# highest difference between reference and any synsitivity, rounded up to next 5er
max_diff = 5 * np.ceil(
    np.max([reference_value - df["-20%"].min(), df["+20%"].max() - reference_value]) / 5
)

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.invert_yaxis()

# Spacing between horizontal lines
h = 0.4

# central vertical black line
ax.plot([reference_value, reference_value], [-1, len(df) + 1], color="k")

ax.set_yticks(list(df.index))
ax.set_yticklabels([s for s in df["label"].to_list()])
ax.set_xlim([reference_value - max_diff, reference_value + max_diff])
ax.set_ylim(0 - h, len(df) - 1 + h)
ax.set_xlabel("Cost per MWh [EUR]")

# additional upper/right axes
axx = ax.twinx()
axx.set_yticks(list())
axx.set_ylim(ax.get_ylim())
axy = ax.twiny()

# Construct upper x-axis label positions and values (in percentage, 10% steps)
lower_percentage_value = (
    np.floor(((1 - max_diff / reference_value) * 100) / 10) * 10
)  # round to include next lower 5%
# since lower percentage value is rounded down, include here next upper 10% as well
upper_percentage_value = (100 - lower_percentage_value) + 100 + 10
axy.set_xticks(
    np.arange(lower_percentage_value, upper_percentage_value, 10)
    * reference_value
    / 100
)
axy.set_xticklabels(
    [f"{v:.0f}%" for v in np.arange(lower_percentage_value, upper_percentage_value, 10)]
)
axy.set_xlim(ax.get_xlim())

# Left/Right side indicator for +/- 20%
ax.text(
    0.05,
    0.5,
    "Parameter -20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)
axx.text(
    0.95,
    0.5,
    "Parameter +20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)

for idx, row in df.iterrows():
    ax.barh(
        y=idx,
        left=row["-20%"],
        width=reference_value - row["-20%"],
        height=h,
        fc="gray",
    )
    ax.barh(
        y=idx,
        left=reference_value,
        width=row["+20%"] - reference_value,
        height=h,
        fc="gray",
    )
    ax.text(
        x=reference_value * 0.995,
        y=idx,
        s="{s:+.1f}%".format(s=(row["-20%"] - reference_value) / reference_value * 100),
        ha="right",
        va="center",
    )
    ax.text(
        x=reference_value * 1.005,
        y=idx,
        s="{s:+.1f}%".format(s=(row["+20%"] - reference_value) / reference_value * 100),
        ha="left",
        va="center",
    )

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])